### Notebook to run and test RAG with chat history (v2)

### Step 1: Initialize notebook

In [1]:
import os 
import bs4
import getpass 
import numpy as np
import faiss

import pickle
from langchain_openai import ChatOpenAI, OpenAIEmbeddings 
from langchain_core.tools import tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import MessagesState, StateGraph
from langgraph.graph import END

from langgraph.checkpoint.memory import MemorySaver
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

from utils import *


USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/matt/miniconda3/envs/py311_DeepBark/lib/python3.11/site-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [25]:
import pickle
import numpy as np

# Load the chunked_documents from the pickle file
with open("../app/vector_storage/chunked_documents.pkl", "rb") as f:
    chunked_documents = pickle.load(f)

# Iterate over each document and convert the metadata 'documents' key from ndarray to list
for doc in chunked_documents:
    if hasattr(doc, "metadata") and "documents" in doc.metadata:
        if isinstance(doc.metadata["documents"], np.ndarray):
            doc.metadata["documents"] = doc.metadata["documents"].tolist()

# Optionally, save the updated documents back to a new file
with open("../app/vector_storage/chunked_documents_updated.pkl", "wb") as f:
    pickle.dump(chunked_documents, f)

print("All numpy arrays in metadata['documents'] have been converted to lists.")


All numpy arrays in metadata['documents'] have been converted to lists.


### Step 2: Initialize model objects

In [2]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [4]:
# Load the FAISS index and documents
index = faiss.read_index("../app/vector_storage/faiss_index.index")

with open("../app/vector_storage/chunked_documents_updated.pkl", "rb") as f:
    all_splits = pickle.load(f)

### Initialize RAG

In [5]:
custom_rag_prompt = PromptTemplate.from_template(prompt_template)

In [6]:
graph_builder = StateGraph(MessagesState)

In [7]:
memory = MemorySaver()

In [8]:
all_splits[0]#.page_content#.content

Document(metadata={'source': 'https://www.dkk.dk/race/affenpinscher', 'specs': {'Aktivitesniveau': 'Mellem', 'FCI Gruppe': 'Gruppe 2 – Schnauzere, pinschere, molosser og sennenhunde', 'Farver': 'Sort, men brune eller grå aftegninger accepteres.', 'Hjemland': 'Tyskland.', 'Højde': '25-30 cm for begge køn.', 'Internationalt racenavn': 'Affenpinscher.', 'Pelspleje': 'Lille', 'Specialklub': 'Dansk Schnauzer Og Pinscher Klub', 'Størrelse': 'Lille', 'Temperament': 'Samarbejdende', 'Vægt': '3-4 kg.'}, 'documents': ['https://www.hundeweb.dk/dkk/public/getRaseFil?FIL_ID=242', 'https://www.hundeweb.dk/dkk/public/getRaseFil?FIL_ID=951', 'https://www.hundeweb.dk/dkk/public/getPasningsVeilederFil?kategori=2'], 'breed_name': 'affenpinscher', 'scrape_timestamp': Timestamp('2025-02-17 09:58:57.507542'), 'content_type': 'dog_breed_profile', 'start_index': 0}, page_content='Breed Profile: Affenpinscher\n## Key Characteristics:\n- Aktivitesniveau: Mellem\n- FCI Gruppe: Gruppe 2 – Schnauzere, pinschere, m

In [9]:
@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    # Create embeddings for the query
    query_embedding = embeddings.embed_query(query)
    query_embedding = np.array([query_embedding])

    distances, indices = index.search(query_embedding, k=3)
    retrieved_docs = []
    for distance, idx in zip(distances[0], indices[0]):
        if distance < 1.0:
            doc = all_splits[idx]
            # Convert to a dictionary format similar to LangChain's Document class
            retrieved_docs.append({
                "metadata": doc.metadata,
                "page_content": doc.page_content
            })
    serialized = "\n\n".join(
        (f"Source: {doc['metadata']['source']}\n" f"Content: {doc['page_content']}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [10]:
# Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}

In [11]:
# Executre the retrieval tool
tools = ToolNode([retrieve])

In [12]:
#Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]
    
    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "Du er en assistent for spørsmålsbesvarende oppgaver. "
        "Bruk følgende bider af hentet kontekst for at svare "
        "på spørsmålet. Hvis du ikke er 100% sikker på svaret,"
        "skal du inkludere et kald til retrieval tool."
        "Hvis du stadigvæk ikke kender svarer skal du svare: Jeg kender ikke svaret."
        "Brug maksimalt tre sætninger og hold "
        "svaret kortfattet."
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [13]:
# build the graph
graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

# set the entry point
graph_builder.set_entry_point("query_or_respond")
# add conditions for passing from one node to another
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {
    END: END, # if the condition indicates the end of the process, terminate
    "tools": "tools" # else, proceed to the tools node
     },
)
# add edges to the graph that connect the nodes
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)
# # compile the graph
# graph = graph_builder.compile()
# initiate the memory saver to save the state of the graph
graph = graph_builder.compile(checkpointer=memory)

# Specify an ID for the thread
config = {"configurable": {"thread_id": "abc123"}}

In [14]:
input_message = "Er en Labrador Retriever en aktiv hund?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Er en Labrador Retriever en aktiv hund?
================================== Ai Message ==================================

Ja, Labrador Retriever er en aktiv hunderase. De er kjent for å ha høy energi og behov for regelmessig mosjon og mental stimulering. Labradore er svært lekne, kjærlige og sosiale hunder som trives med aktiviteter som lange turer, svømming, apportering og forskjellige hundesporter. De er også kjent for å være gode familiehunder på grunn av sin vennlige og tålmodige natur.


In [15]:
input_message = "Kræver en labrador retriever meget pelspleje?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Kræver en labrador retriever meget pelspleje?
================================== Ai Message ==================================

Labrador Retriever har en kort, tett pels som er relativt lavpåvirket når det gjelder pelspleie. Dog krever de en vis grad af pleje for at holde pelsen sund og fri for snavs og tangle. Her er nogle nøglepunkter om pelspleje for Labradorer:

1. **Børstning**: Det anbefales at børste en Labrador cirka en gang om ugen for at fjerne døde hår og reducere fældningen. Under fældningssæsonen (forår og efterår) kan det være nødvendigt at børste dem oftere.

2. **Badning**: Labradore har ikke brug for hyppige bade, da deres pels har naturlige olier, der beskytter huden. Badning en gang hver 1-3 måned er som regel tilstrækkeligt, medmindre hunden bliver meget snavset.

3. **Ører og kløer**: Som med alle hunde er det vigtigt at holde øje med ørenes sundhed og trimme kløerne regelmæssigt.

Al

In [16]:
input_message = "Er en Finsk lapphund mere eller mindre aktiv end en labrador retriever?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Er en Finsk lapphund mere eller mindre aktiv end en labrador retriever?
================================== Ai Message ==================================

En Finsk Lapphund og en Labrador Retriever er begge aktive hunderacer, men deres aktivitetsniveau og behov kan variere noget.

- **Labrador Retriever**: Generelt betragtes de som en af de mest aktive hunderacer. De har et højt energiniveau og kræver meget motion og mental stimulering. De elsker at svømme, lege apport, og deltage i forskellige aktiviteter.

- **Finsk Lapphund**: Denne race er også aktiv, men de har tendens til at have lidt lavere energi sammenlignet med Labradore. Finske Lapphunde er arbejdende hunde, der historisk set blev brugt til at vogte og føre rensdyr. De nyder motion, men de kan også være mere uafhængige og mindre krævende i forhold til konstant aktivitet og stimulation.

Sammenfattende kan man sige, at en Labrador Retriever gener

In [17]:
input_message = "Hvilke farver er tilladt for en Finsk Lapphund?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hvilke farver er tilladt for en Finsk Lapphund?
================================== Ai Message ==================================

Finske Lapphunde findes i flere forskellige farver, som er godkendt af de fleste kennelklubber. De tilladte farver omfatter:

1. **Sort**: En ensfarvet sort pels er meget almindelig.
2. **Brun**: En ensfarvet brun farve er også accepteret.
3. **Grå**: En grå pelskombination, som ofte kan være med eller uden sorte afmærkninger.
4. **Hvid**: Hvid eller næsten hvid pels er tilladt.
5. **Rød**: Rød eller lys rød farve kan også forekomme.
6. **Sabel (fawn)**: En lysere, gylden farve henholdsvis.
   
Der kan også forekomme forskellige afmærkninger og mønstre, såsom hvidt på brystet og poter, som er almindeligt i racen. Det er altid bedst at konsultere den specifikke kennelklub eller standarden for at få den mest præcise information om farver og mønstre.


In [18]:
input_message = "Hvilke farver mener DKK at der er tilladte for en Finsk Lapphund?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hvilke farver mener DKK at der er tilladte for en Finsk Lapphund?
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_2LK9kpBmNAe7mIAp0PjpHtuG)
 Call ID: call_2LK9kpBmNAe7mIAp0PjpHtuG
  Args:
    query: Finske Lapphund farver site:dkk.dk
================================= Tool Message =================================
Name: retrieve

Source: https://www.dkk.dk/race/finsk-lapphund
Content: Breed Profile: Finsk-lapphund
## Key Characteristics:
- Aktivitesniveau: Mellem
- FCI Gruppe: Gruppe 5 – Spidshunde
- Farver: Alle farver er tilladt.
- Hjemland: Finland.
- Højde: Hanner 46-52 cm. Tæver 41-47 cm.
- Internationalt racenavn: Suomenlapinkoira.
- Pelspleje: Højt
- Specialklub: Spidshundeklubben
- Størrelse: Mellem
- Temperament: Selvstendig
- Vægt: 19-21 kg.
## Detailed Description:
Mentalitet
En finsk lapphund har med det stabile væsen stadig mange af

In [ ]:
input_message = "Er en Labrador Retriever en aktiv hund?"

query = {"messages": [{"role": "user", "content": input_message}]}

output = graph.stream(query, stream_mode="values", config=config)

result = [step["messages"] for step in output][-1]

In [15]:
graph.get_state(config=config).values['messages'] 

[HumanMessage(content='Er en Labrador Retriever en aktiv hund?', additional_kwargs={}, response_metadata={}, id='b01b0521-3c64-454c-af8a-4e1046b90c72'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_69QxuOwCbSXPr7jKJgPBylmt', 'function': {'arguments': '{"query":"Er en Labrador Retriever en aktiv hund?"}', 'name': 'retrieve'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 51, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2e1bde45-9715-4e5d-b6c7-97d448fb0ee4-0', tool_calls=[{'name': 'retrieve', 'args': {'query': 'Er en Labrador Retriever en aktiv hund?'}, 'id': 'call_69QxuOwCbSXPr7jK

In [16]:
from langchain_core.messages import ToolMessage

last_tool_message = next(
        (
            tool_msg for tool_msg in reversed(result)
            if isinstance(tool_msg, ToolMessage)
        )
    )

"\n\n".join(doc.content for doc in [last_tool_message])

StopIteration: 

In [17]:
input_message_2 = "Hvad spiste Adolf Hitler under krigen?"

query_2 = {"messages": [{"role": "user", "content": input_message_2}]}

output_2 = graph.stream(query_2, stream_mode="values", config=config)

result_2 = [step["messages"] for step in output_2][-1]

1.4002593
1.4171115
1.4468055


In [18]:
# get all 
graph.get_state(config=config).values['messages'] #.messages

[HumanMessage(content='Er en labrador en aktiv hund?', additional_kwargs={}, response_metadata={}, id='0d572558-10f8-4641-a18e-76b48b00c75b'),
 AIMessage(content='Ja, labradorer er generelt aktive hunder. De er kjent for sitt vennlige lynne og høye energinivå. Labradorer trenger regelmessig mosjon og mentale stimuleringsaktiviteter for å holde seg sunne og glade. De er ofte engasjert i aktiviteter som jakt, svømming, og forskjellige hundesporter.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 52, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1', 'finish_reason': 'stop', 'logprobs': None}, id='run-afeac389-b78b-4ded-a5fb-4704bd42d435-0', usage_metadata={'input_tokens'

In [ ]:
response = res[-1]

final_response = res[-1][-1]

In [34]:
from langchain_core.messages import ToolMessage

answer = final_response.content
# Extract sources from ToolMessages
sources = []
for step in res:
    for message in step:
        if isinstance(message, ToolMessage):
            sources.extend([doc.metadata["source"] for doc in message.artifact])
sources = set(sources)

In [47]:
res[-1][2].artifact[0].metadata

{'source': 'https://petguide.dk/hundefoder-maerker/',
 'start_index': 2355,
 'section': 'beginning'}

In [36]:
sources = [doc.metadata["source"] for doc in final_response.tool_calls[0].artifact]


IndexError: list index out of range

In [16]:
input_message = "Hvilke fodermærker kommer fra Canada?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hvilke fodermærker kommer fra Canada?
================================== Ai Message ==================================
Tool Calls:
  retrieve (call_tTYrxjL4JmHyUV1txNlE90de)
 Call ID: call_tTYrxjL4JmHyUV1txNlE90de
  Args:
    query: fodermærker fra Canada
================================= Tool Message =================================
Name: retrieve

Source: {'source': 'https://petguide.dk/hundefoder-maerker/', 'start_index': 2355, 'section': 'beginning'}
Content: 1st Choice
1st Choice er kvalitetsfoder lavet i Canada, af gode lokale råvarer skabt med det formål, at sikre at hunde får den rigtige ernæring gennem hele deres liv. Med special udviklede formler, sørger de for at hundens behov bliver opfyldt gennem hundens forskellige livsfaser. Velsmag er i højsæde, men der er samtidigt gjort ekstra ud af at sikre at hunden får ekstra energi, kontrolleret sin vægt, sund hud og pels. 1st Choices vigtigste form